Perform residual RAKI reconstruction on a variety of ACS sizes and accelerations (similar for what I did in SPARK and RAKI, for sake of experimental comparison)

In [1]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
import scipy.io as sio
import numpy as np
import numpy.matlib
import time
import os

from utils import signalprocessing as sig

Instructions for updating:
non-resource variables are not supported in the long term


### RAKI Definitions 

In [2]:
def weight_variable(shape,vari_name):                   
    initial = tf.truncated_normal(shape, stddev=0.1,dtype=tf.float32)
    return tf.Variable(initial,name = vari_name)

def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape,dtype=tf.float32)
    return tf.Variable(initial)

def conv2d_same(x,W):
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

def conv2d_dilate_same(x, W,dilate_rate):
    return tf.nn.convolution(x, W,padding='SAME',dilation_rate = [1,dilate_rate])

def conv2d(x, W):
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='VALID')

def conv2d_dilate(x, W,dilate_rate):
    return tf.nn.convolution(x, W,padding='VALID',dilation_rate = [1,dilate_rate])

def learning_residual_raki(ACS_input,target_input,accrate_input,sess,\
    ACS_dim_X,ACS_dim_Y,ACS_dim_Z,target_dim_X,target_dim_Y,target_dim_Z,\
    target,target_x_start,target_x_end,target_y_start,target_y_end,\
    ACS):
    input_ACS    = tf.placeholder(tf.float32, [1, ACS_dim_X,ACS_dim_Y,ACS_dim_Z])                                  
    input_Target = tf.placeholder(tf.float32, [1, target_dim_X,target_dim_Y,target_dim_Z]) 
    
    Input = tf.reshape(input_ACS, [1, ACS_dim_X, ACS_dim_Y, ACS_dim_Z])         

    [target_dim0,target_dim1,target_dim2,target_dim3] = np.shape(target)
    
    W_conv1 = weight_variable([kernel_x_1, kernel_y_1, ACS_dim_Z, layer1_channels],'W1') 
    h_conv1 = tf.nn.relu(conv2d_dilate(Input, W_conv1,accrate_input)) 

    W_conv2 = weight_variable([kernel_x_2, kernel_y_2, layer1_channels, layer2_channels],'W2')
    h_conv2 = tf.nn.relu(conv2d_dilate(h_conv1, W_conv2,accrate_input))

    W_conv3 = weight_variable([kernel_last_x, kernel_last_y, layer2_channels, target_dim3],'W3')
    h_conv3 = conv2d_dilate(h_conv2, W_conv3,accrate_input)
    
    W_conv_linear = weight_variable([kernel_x_linear,kernel_y_linear,ACS_dim_Z,target_dim3],'W_lin')
    h_linear = conv2d_dilate(Input,W_conv_linear,accrate_input)
    x_length = h_conv3.shape[1]
    y_length = h_conv3.shape[2]
    
    if(y_length % 2 == 0):
        h_linear = h_linear[:,x_length//2 - x_length//2:x_length//2 + x_length//2,\
            y_length//2 - y_length//2:y_length//2 + y_length//2,:]
    else:
        h_linear = h_linear[:,x_length//2 - x_length//2:x_length//2 + x_length//2,\
            y_length//2 - y_length//2:y_length//2 + y_length//2+1,:]
        
    error_norm = tf.norm(input_Target - h_linear) + tf.norm(input_Target - h_linear - h_conv3)
    train_step = tf.train.AdamOptimizer(LearningRate).minimize(error_norm)
    
     
    if int((tf.__version__).split('.')[1]) < 12 and int((tf.__version__).split('.')[0]) < 1:
        init = tf.initialize_all_variables()
    else:
        init = tf.global_variables_initializer()
    sess.run(init)

    error_prev = 1 
    for i in range(MaxIteration):
        
        sess.run(train_step, feed_dict={input_ACS: ACS, input_Target: target})
        if i % 100 == 0:                                                                      
            error_now=sess.run(error_norm,feed_dict={input_ACS: ACS, input_Target: target})    
            print('The',i,'th iteration gives an error',error_now)  
            
    error = sess.run(error_norm,feed_dict={input_ACS: ACS, input_Target: target})
    return [sess.run(W_conv1),sess.run(W_conv2),sess.run(W_conv3),sess.run(W_conv_linear),error]  
    
def cnn_linear(input_kspace,w_linear,acc_rate,sess):
    return sess.run(conv2d_dilate(input_kspace,w_linear,acc_rate))

def cnn_3layer(input_kspace,w1,w2,w3,acc_rate,sess):                
    h_conv1 = tf.nn.relu(conv2d_dilate(input_kspace, w1,acc_rate)) 
    h_conv2 = tf.nn.relu(conv2d_dilate(h_conv1, w2,acc_rate))
    h_conv3 = conv2d_dilate(h_conv2, w3,acc_rate) 
    return sess.run(h_conv3)           

### Loading k-space and defining fft operators

In [3]:
fft2c_raki  = lambda x: sig.fft(sig.fft(x,0),1)
ifft2c_raki = lambda x: sig.ifft(sig.ifft(x,0),1)

image_coils_truth  = sio.loadmat('data/img_grappa_32chan.mat')['IMG']

kspace_truth_raki  = fft2c_raki(image_coils_truth)
[M,N,C] = kspace_truth_raki.shape

### Setting Non-changing Experimental Parameters 

In [4]:
GPU_FRAC = 1/4

Rx     = 1
acsx   = M

#### Linear Network Parameters ####
kernel_x_linear = 5
kernel_y_linear = 2

#### RAKI Network Parameters ####
kernel_x_1 = 5
kernel_y_1 = 2

kernel_x_2 = 1
kernel_y_2 = 1

kernel_last_x = 3
kernel_last_y = 2

layer1_channels = 32 
layer2_channels = 8

MaxIteration = 1000
LearningRate = 3e-3  

### Setting ablation parameters

In [5]:
all_Ry    = [5,6]
all_acsy  = [20,24,30,36,40]
             
all_parameters = []

for ry in all_Ry:
    for acsy in all_acsy:
        all_parameters.append({'Ry':ry, 'acsy':acsy})

### Defining residual-RAKI reconstruction function (which will be looped over the parameters)

In [6]:
def residual_raki(MaxIteration,LearningRate,Rx,Ry,acsx,acsy,GPU_FRAC,\
    kspace_truth_raki=kspace_truth_raki,kernel_x_linear=kernel_x_linear,\
    kernel_y_linear=kernel_y_linear,kernel_x_1=kernel_x_1,kernel_y_1=kernel_y_1,kernel_x_2=kernel_x_2,\
    kernel_y_2=kernel_y_2,kernel_last_x=kernel_last_x,kernel_last_y=kernel_last_y,
    layer1_channels=layer1_channels,layer2_channels=layer2_channels):
    
    acsregionX = np.arange(M//2 - acsx // 2,M//2 + acsx//2) 
    acsregionY = np.arange(N//2 - acsy // 2,N//2 + acsy//2) 

    kspace_raki_undersampled_withacs = np.zeros((M,N,C),dtype = complex)
    kspace_raki_undersampled_withacs[::Rx,::Ry,:] = kspace_truth_raki[::Rx,::Ry,:]
    kspace_raki_undersampled_withacs[acsregionX[0]:acsregionX[acsx-1]+1,acsregionY[0]:acsregionY[acsy-1]+1,:]\
        = kspace_truth_raki[acsregionX[0]:acsregionX[acsx-1]+1,acsregionY[0]:acsregionY[acsy-1]+1,:]

    kspace = np.copy(kspace_raki_undersampled_withacs)
    no_ACS_flag = 0;
    normalize = 0.015/np.max(abs(kspace[:]))
    kspace = np.multiply(kspace,normalize) 

    [m1,n1,no_ch] = np.shape(kspace)
    no_inds = 1

    kspace_all = np.copy(kspace);
    kx = np.transpose(np.int32([(range(1,m1+1))]))                  
    ky = np.int32([(range(1,n1+1))])

    kspace = np.copy(kspace_all)
    mask = np.squeeze(np.matlib.sum(np.matlib.sum(np.abs(kspace),0),1))>0; 
    picks = np.where(mask == 1);                                  
    kspace = kspace[:,np.int32(picks[0][0]):n1+1,:]
    kspace_all = kspace_all[:,np.int32(picks[0][0]):n1+1,:]  

    kspace_NEVER_TOUCH = np.copy(kspace_all)

    mask = np.squeeze(np.matlib.sum(np.matlib.sum(np.abs(kspace),0),1))>0;  
    picks = np.where(mask == 1);                                  
    d_picks = np.diff(picks,1)  
    indic = np.where(d_picks == 1);

    mask_x = np.squeeze(np.matlib.sum(np.matlib.sum(np.abs(kspace),2),1))>0;
    picks_x = np.where(mask_x == 1);
    x_start = picks_x[0][0]
    x_end = picks_x[0][-1]


    no_ACS_flag=0;
    print('ACS signal found in the input data')
    indic = indic[1][:]
    center_start = picks[0][indic[0]];
    center_end = picks[0][indic[-1]+1];
    ACS = kspace[x_start:x_end+1,center_start:center_end+1,:]
    [ACS_dim_X, ACS_dim_Y, ACS_dim_Z] = np.shape(ACS)
    ACS_re = np.zeros([ACS_dim_X,ACS_dim_Y,ACS_dim_Z*2])
    ACS_re[:,:,0:no_ch] = np.real(ACS)
    ACS_re[:,:,no_ch:no_ch*2] = np.imag(ACS)

    acc_rate = d_picks[0][0]
    no_channels = ACS_dim_Z*2

    time_ALL_start = time.time()

    [ACS_dim_X, ACS_dim_Y, ACS_dim_Z] = np.shape(ACS_re)
    ACS = np.reshape(ACS_re, [1,ACS_dim_X, ACS_dim_Y, ACS_dim_Z]) 
    ACS = np.float32(ACS)  
    
    w_linear_all = \
        np.zeros([kernel_x_linear, kernel_y_linear, no_channels, acc_rate - 1, no_channels],dtype=np.float32)
        
    w1_all = np.zeros([kernel_x_1, kernel_y_1, no_channels, layer1_channels, no_channels],dtype=np.float32)
    w2_all = np.zeros([kernel_x_2, kernel_y_2, layer1_channels,layer2_channels,no_channels],dtype=np.float32)
    w3_all = np.zeros([kernel_last_x, kernel_last_y, layer2_channels,acc_rate - 1, no_channels],dtype=np.float32)    

    target_x_start = np.int32(np.ceil(kernel_x_1/2) + np.floor(kernel_x_2/2) + np.floor(kernel_last_x/2) -1); 
    target_x_end = np.int32(ACS_dim_X - target_x_start -1); 

    target_y_start = np.int32((np.ceil(kernel_y_1/2)-1) + (np.ceil(kernel_y_2/2)-1) + (np.ceil(kernel_last_y/2)-1)) * acc_rate;     
    target_y_end = ACS_dim_Y  - np.int32((np.floor(kernel_y_1/2) + np.floor(kernel_y_2/2) + np.floor(kernel_last_y/2))) * acc_rate -1;

    target_dim_X = target_x_end - target_x_start + 1
    target_dim_Y = target_y_end - target_y_start + 1
    target_dim_Z = acc_rate - 1
    
    print('go!')
    time_Learn_start = time.time() 

    errorSum = 0;
    config = tf.ConfigProto()
    config.gpu_options.per_process_gpu_memory_fraction = GPU_FRAC ; 

    for ind_c in range(ACS_dim_Z):

        sess = tf.Session(config=config)
        # set target lines
        target = np.zeros([1,target_dim_X,target_dim_Y,target_dim_Z])
        print('learning channel #',ind_c+1)
        time_channel_start = time.time()

        for ind_acc in range(acc_rate-1):
            target_y_start = np.int32((np.ceil(kernel_y_1/2)-1) + (np.ceil(kernel_y_2/2)-1) + (np.ceil(kernel_last_y/2)-1)) * acc_rate + ind_acc + 1 
            target_y_end = ACS_dim_Y  - np.int32((np.floor(kernel_y_1/2) + (np.floor(kernel_y_2/2)) + np.floor(kernel_last_y/2))) * acc_rate + ind_acc
            target[0,:,:,ind_acc] = ACS[0,target_x_start:target_x_end + 1, target_y_start:target_y_end +1,ind_c];

        # learning

        [w1,w2,w3,w_linear,error]=learning_residual_raki(ACS,target,acc_rate,sess,\
            ACS_dim_X,ACS_dim_Y,ACS_dim_Z,target_dim_X,target_dim_Y,target_dim_Z,target,\
            target_x_start,target_x_end,target_y_start,target_y_end,\
            ACS) 
        w1_all[:,:,:,:,ind_c] = w1
        w2_all[:,:,:,:,ind_c] = w2
        w3_all[:,:,:,:,ind_c] = w3     
        w_linear_all[:,:,:,:,ind_c] = w_linear
        
        time_channel_end = time.time()
        print('Time Cost:',time_channel_end-time_channel_start,'s')
        print('Norm of Error = ',error)
        errorSum = errorSum + error

        sess.close()
        tf.reset_default_graph()

    time_Learn_end = time.time();

    print('lerning step costs:',(time_Learn_end - time_Learn_start)/60,'min')    
    kspace_recon_all = np.copy(kspace_all)
    kspace_recon_all_nocenter = np.copy(kspace_all)

    kspace = np.copy(kspace_all)

    over_samp = np.setdiff1d(picks,np.int32([range(0, n1,acc_rate)]))
    kspace_und = kspace
    kspace_und[:,over_samp,:] = 0;
    [dim_kspaceUnd_X,dim_kspaceUnd_Y,dim_kspaceUnd_Z] = np.shape(kspace_und)

    kspace_und_re = np.zeros([dim_kspaceUnd_X,dim_kspaceUnd_Y,dim_kspaceUnd_Z*2])
    kspace_und_re[:,:,0:dim_kspaceUnd_Z] = np.real(kspace_und)
    kspace_und_re[:,:,dim_kspaceUnd_Z:dim_kspaceUnd_Z*2] = np.imag(kspace_und)
    kspace_und_re = np.float32(kspace_und_re)
    kspace_und_re = np.reshape(kspace_und_re,[1,dim_kspaceUnd_X,dim_kspaceUnd_Y,dim_kspaceUnd_Z*2])
    kspace_recon = kspace_und_re

    kspace_recon_linear       = np.copy(kspace_recon)
    kspace_recon_residual     = np.zeros(kspace_recon.shape,dtype = complex)
        
    config = tf.ConfigProto()
    config.gpu_options.per_process_gpu_memory_fraction = GPU_FRAC ; 

    for ind_c in range(0,no_channels):
        print('Reconstruting Channel #',ind_c+1)

        sess = tf.Session(config=config) 
        if int((tf.__version__).split('.')[1]) < 12 and int((tf.__version__).split('.')[0]) < 1:
            init = tf.initialize_all_variables()
        else:
            init = tf.global_variables_initializer()
        sess.run(init)

        # grab w and b
        w1 = np.float32(w1_all[:,:,:,:,ind_c])
        w2 = np.float32(w2_all[:,:,:,:,ind_c])     
        w3 = np.float32(w3_all[:,:,:,:,ind_c])

        w_linear = np.float32(w_linear_all[:,:,:,:,ind_c])
        
        residual_recon = cnn_3layer(kspace_und_re,w1,w2,w3,acc_rate,sess) 
        linear_recon   = cnn_linear(kspace_und_re,w_linear,acc_rate,sess)  
        x_length = residual_recon.shape[1]
        y_length = residual_recon.shape[2]
        linear_recon = linear_recon[:,x_length//2 - x_length//2:x_length//2 + x_length//2,\
                y_length//2 - y_length//2:y_length//2 + y_length//2,:]
        #linear_recon = np.copy(residual_recon)
        
        target_x_end_kspace = dim_kspaceUnd_X - target_x_start;            
        
        for ind_acc in range(0,acc_rate-1):
            target_y_start = \
                np.int32((np.ceil(kernel_y_1/2)-1) + np.int32((np.ceil(kernel_y_2/2)-1)) + \
                np.int32(np.ceil(kernel_last_y/2)-1)) * acc_rate + ind_acc + 1; 
            
            target_y_end_kspace = \
                dim_kspaceUnd_Y - np.int32((np.floor(kernel_y_1/2)) + (np.floor(kernel_y_2/2)) + np.floor(kernel_last_y/2)) * acc_rate + ind_acc;
               
            kspace_recon[0,target_x_start:target_x_end_kspace,target_y_start:target_y_end_kspace+1:acc_rate,ind_c] = \
                linear_recon[0,:,::acc_rate,ind_acc] + residual_recon[0,:,::acc_rate,ind_acc];
            kspace_recon_linear[0,target_x_start:target_x_end_kspace,target_y_start:target_y_end_kspace+1:acc_rate,ind_c] \
                = linear_recon[0,:,::acc_rate,ind_acc];
            kspace_recon_residual[0,target_x_start:target_x_end_kspace,target_y_start:target_y_end_kspace+1:acc_rate,ind_c] = \
                residual_recon[0,:,::acc_rate,ind_acc]
            
    kspace_recon          = np.squeeze(kspace_recon)
    kspace_recon_linear   = np.squeeze(kspace_recon_linear)
    kspace_recon_residual = np.squeeze(kspace_recon_residual)
    
    kspace_recon_complex = (kspace_recon[:,:,0:np.int32(no_channels/2)] + \
                np.multiply(kspace_recon[:,:,np.int32(no_channels/2):no_channels],1j))

    kspace_recon_complex_linear = (kspace_recon_linear[:,:,0:np.int32(no_channels/2)] + \
                np.multiply(kspace_recon_linear[:,:,np.int32(no_channels/2):no_channels],1j))

    kspace_recon_complex_residual = (kspace_recon_residual[:,:,0:np.int32(no_channels/2)] + \
                np.multiply(kspace_recon_residual[:,:,np.int32(no_channels/2):no_channels],1j))
    
    kspace_recon_all_nocenter[:,:,:] = np.copy(kspace_recon_complex); 


    if no_ACS_flag == 0: 
        kspace_recon_complex[:,center_start:center_end,:] = kspace_NEVER_TOUCH[:,center_start:center_end,:]
        
        kspace_recon_complex_linear[:,center_start:center_end,:] = \
            kspace_NEVER_TOUCH[:,center_start:center_end,:]
        
        #kspace_recon_complex_residaul[:,center_start:center_end,:] = \
        #    kspace_NEVER_TOUCH[:,center_start:center_end,:]
        print('ACS signal has been putted back')
    else:
        print('No ACS signal is putted into k-space')

        sess.close()
        tf.reset_default_graph()
        
    return [kspace_recon_complex,kspace_recon_complex_linear,kspace_recon_complex_residual]

### Defining loop over the reconstruction parameters

In [ ]:
kspace_rraki_recon_all  = np.zeros((M,N,C,len(all_parameters)),dtype = complex)
kspace_linear_recon_all = np.zeros((M,N,C,len(all_parameters)),dtype = complex)
kspace_residual_est_all = np.zeros((M,N,C,len(all_parameters)),dtype = complex)

for index,parameter in enumerate(all_parameters):
    Ry   = parameter['Ry']
    acsy = parameter['acsy']
    
    print('~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~')
    print('Training %d/%d || Ry %d || Acsy %d' % (index+1,len(all_parameters),Ry,acsy))
    print('~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~')
    
    [kspace_rraki,kspace_linear,kspace_residual] = \
        residual_raki(MaxIteration,LearningRate,Rx,Ry,acsx,acsy,GPU_FRAC)
    
    kspace_rraki_recon_all[:,:,:,index]     = kspace_rraki
    kspace_linear_recon_all[:,:,:,index]    = kspace_linear
    kspace_residual_est_all[:,:,:,index]    = kspace_residual

~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Training 1/10 || Ry 5 || Acsy 20
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
ACS signal found in the input data
go!
learning channel # 1
The 0 th iteration gives an error 0.097926065
The 100 th iteration gives an error 0.0043357518
The 200 th iteration gives an error 0.0030820537
The 300 th iteration gives an error 0.0027214957
The 400 th iteration gives an error 0.0024870643
The 500 th iteration gives an error 0.0023155846
The 600 th iteration gives an error 0.002193783
The 700 th iteration gives an error 0.0020632278
The 800 th iteration gives an error 0.0020019389
The 900 th iteration gives an error 0.0019461946
Time Cost: 22.51112389564514 s
Norm of Error =  0.0018800918
learning channel # 2
The 0 th iteration gives an error 0.0968244
The 100 th iteration gives an error 0.004286646
The 200 th iteration gives an error 0.0031334567
The 300 th iteration gives an error 0.0026511608
The 400 th iteration gives an error 0.0024230434
The 500 th iteration gives an 

The 200 th iteration gives an error 0.0032335527
The 300 th iteration gives an error 0.0027209176
The 400 th iteration gives an error 0.0025116487
The 500 th iteration gives an error 0.0023204177
The 600 th iteration gives an error 0.002125999
The 700 th iteration gives an error 0.0020450675
The 800 th iteration gives an error 0.0019955514
The 900 th iteration gives an error 0.0018803765
Time Cost: 3.7203831672668457 s
Norm of Error =  0.0018210849
learning channel # 16
The 0 th iteration gives an error 0.094818875
The 100 th iteration gives an error 0.0042658933
The 200 th iteration gives an error 0.0030040387
The 300 th iteration gives an error 0.0025639571
The 400 th iteration gives an error 0.0022828286
The 500 th iteration gives an error 0.0021109695
The 600 th iteration gives an error 0.0020969552
The 700 th iteration gives an error 0.0018818721
The 800 th iteration gives an error 0.0018544466
The 900 th iteration gives an error 0.001781741
Time Cost: 3.628878593444824 s
Norm of 

The 800 th iteration gives an error 0.0018641276
The 900 th iteration gives an error 0.0017924537
Time Cost: 3.501310348510742 s
Norm of Error =  0.0017143604
learning channel # 30
The 0 th iteration gives an error 0.08577026
The 100 th iteration gives an error 0.0036119618
The 200 th iteration gives an error 0.002618562
The 300 th iteration gives an error 0.0022293706
The 400 th iteration gives an error 0.0020263724
The 500 th iteration gives an error 0.0018572756
The 600 th iteration gives an error 0.001735812
The 700 th iteration gives an error 0.0016404407
The 800 th iteration gives an error 0.0016143199
The 900 th iteration gives an error 0.001512794
Time Cost: 3.4249231815338135 s
Norm of Error =  0.0014801838
learning channel # 31
The 0 th iteration gives an error 0.12071874
The 100 th iteration gives an error 0.00532691
The 200 th iteration gives an error 0.0037544812
The 300 th iteration gives an error 0.0032316574
The 400 th iteration gives an error 0.0029546097
The 500 th it

The 200 th iteration gives an error 0.0027992968
The 300 th iteration gives an error 0.0024427155
The 400 th iteration gives an error 0.0021953273
The 500 th iteration gives an error 0.002094791
The 600 th iteration gives an error 0.0019491208
The 700 th iteration gives an error 0.001929954
The 800 th iteration gives an error 0.0018450713
The 900 th iteration gives an error 0.0017725173
Time Cost: 3.7403783798217773 s
Norm of Error =  0.001713966
learning channel # 45
The 0 th iteration gives an error 0.087258175
The 100 th iteration gives an error 0.0037046466
The 200 th iteration gives an error 0.002829038
The 300 th iteration gives an error 0.0024568667
The 400 th iteration gives an error 0.0021909066
The 500 th iteration gives an error 0.0020614972
The 600 th iteration gives an error 0.0020095836
The 700 th iteration gives an error 0.001888745
The 800 th iteration gives an error 0.0018679481
The 900 th iteration gives an error 0.0018490145
Time Cost: 3.602060556411743 s
Norm of Err

The 800 th iteration gives an error 0.0018407204
The 900 th iteration gives an error 0.0018294132
Time Cost: 3.617480993270874 s
Norm of Error =  0.0017510294
learning channel # 59
The 0 th iteration gives an error 0.12184443
The 100 th iteration gives an error 0.007901682
The 200 th iteration gives an error 0.00485009
The 300 th iteration gives an error 0.0039164596
The 400 th iteration gives an error 0.0035016981
The 500 th iteration gives an error 0.0031919042
The 600 th iteration gives an error 0.0030510225
The 700 th iteration gives an error 0.0028451385
The 800 th iteration gives an error 0.0027719946
The 900 th iteration gives an error 0.0026978105
Time Cost: 3.715421676635742 s
Norm of Error =  0.0025466825
learning channel # 60
The 0 th iteration gives an error 0.09480062
The 100 th iteration gives an error 0.0041282787
The 200 th iteration gives an error 0.0029141828
The 300 th iteration gives an error 0.002444772
The 400 th iteration gives an error 0.0021858085
The 500 th it

...

The 600 th iteration gives an error 0.0036035427
The 700 th iteration gives an error 0.0034809383
The 800 th iteration gives an error 0.0033542183
The 900 th iteration gives an error 0.0032516392
Time Cost: 3.5848307609558105 s
Norm of Error =  0.0031710002
learning channel # 26
The 0 th iteration gives an error 0.114642195
The 100 th iteration gives an error 0.005791764
The 200 th iteration gives an error 0.004167781
The 300 th iteration gives an error 0.00350864
The 400 th iteration gives an error 0.0032196997
The 500 th iteration gives an error 0.003019688
The 600 th iteration gives an error 0.0028768168
The 700 th iteration gives an error 0.0027611833
The 800 th iteration gives an error 0.0026602698
The 900 th iteration gives an error 0.0025800061
Time Cost: 3.5839879512786865 s
Norm of Error =  0.0025308616
learning channel # 27
The 0 th iteration gives an error 0.1537525
The 100 th iteration gives an error 0.011252433
The 200 th iteration gives an error 0.0067386013
The 300 th it

The 0 th iteration gives an error 0.093227066
The 100 th iteration gives an error 0.005402837
The 200 th iteration gives an error 0.0044233347
The 300 th iteration gives an error 0.0038517725
The 400 th iteration gives an error 0.0035276562
The 500 th iteration gives an error 0.0033990503
The 600 th iteration gives an error 0.0032440163
The 700 th iteration gives an error 0.003121262
The 800 th iteration gives an error 0.0030444073
The 900 th iteration gives an error 0.0029939564
Time Cost: 3.595466375350952 s
Norm of Error =  0.0029328978
learning channel # 41
The 0 th iteration gives an error 0.11337203
The 100 th iteration gives an error 0.004958974
The 200 th iteration gives an error 0.0037544048
The 300 th iteration gives an error 0.0031809544
The 400 th iteration gives an error 0.002858263
The 500 th iteration gives an error 0.0026727594
The 600 th iteration gives an error 0.0025061332
The 700 th iteration gives an error 0.0024359482
The 800 th iteration gives an error 0.00229851

The 600 th iteration gives an error 0.0027303402
The 700 th iteration gives an error 0.0026050266
The 800 th iteration gives an error 0.0025034384
The 900 th iteration gives an error 0.002421875
Time Cost: 3.5819339752197266 s
Norm of Error =  0.0023563923
learning channel # 55
The 0 th iteration gives an error 0.12749256
The 100 th iteration gives an error 0.0072530983
The 200 th iteration gives an error 0.0051957043
The 300 th iteration gives an error 0.0045479955
The 400 th iteration gives an error 0.0042099967
The 500 th iteration gives an error 0.0039385036
The 600 th iteration gives an error 0.0037807026
The 700 th iteration gives an error 0.0036530173
The 800 th iteration gives an error 0.0035562324
The 900 th iteration gives an error 0.0034948639
Time Cost: 3.5627377033233643 s
Norm of Error =  0.003435742
learning channel # 56
The 0 th iteration gives an error 0.126682
The 100 th iteration gives an error 0.00501201
The 200 th iteration gives an error 0.003627337
The 300 th ite

The 700 th iteration gives an error 0.0035258685
The 800 th iteration gives an error 0.003403699
The 900 th iteration gives an error 0.0033276083
Time Cost: 82.45131301879883 s
Norm of Error =  0.0032414545
learning channel # 2
The 0 th iteration gives an error 0.13551879
The 100 th iteration gives an error 0.006851007
The 200 th iteration gives an error 0.00500216
The 300 th iteration gives an error 0.0042754067
The 400 th iteration gives an error 0.0039428724
The 500 th iteration gives an error 0.0037414865
The 600 th iteration gives an error 0.0035233079
The 700 th iteration gives an error 0.0033554626
The 800 th iteration gives an error 0.0032943378
The 900 th iteration gives an error 0.0031869141
Time Cost: 3.699430465698242 s
Norm of Error =  0.0031312304
learning channel # 3
The 0 th iteration gives an error 0.09693773
The 100 th iteration gives an error 0.006241869
The 200 th iteration gives an error 0.004805576
The 300 th iteration gives an error 0.0042703655
The 400 th iterat

The 200 th iteration gives an error 0.0046577947
The 300 th iteration gives an error 0.004009048
The 400 th iteration gives an error 0.0035618073
The 500 th iteration gives an error 0.0033353134
The 600 th iteration gives an error 0.0031935507
The 700 th iteration gives an error 0.0030880785
The 800 th iteration gives an error 0.0030347242
The 900 th iteration gives an error 0.0028774717
Time Cost: 3.9062306880950928 s
Norm of Error =  0.0028627221
learning channel # 17
The 0 th iteration gives an error 0.10434522
The 100 th iteration gives an error 0.0065837866
The 200 th iteration gives an error 0.0048350045
The 300 th iteration gives an error 0.0042161746
The 400 th iteration gives an error 0.0038797255
The 500 th iteration gives an error 0.0036375104
The 600 th iteration gives an error 0.0035754973
The 700 th iteration gives an error 0.0034261465
The 800 th iteration gives an error 0.003304717
The 900 th iteration gives an error 0.003247411
Time Cost: 3.9077882766723633 s
Norm of E

### Computing all reconstructions and rmse's to the corresponding ground truth 

In [ ]:
all_recons_rraki      = np.transpose(sig.rsos(ifft2c_raki(kspace_rraki_recon_all),-2),(1,0,2))
all_recons_linear     = np.transpose(sig.rsos(ifft2c_raki(kspace_linear_recon_all),-2),(1,0,2))
all_est_residual     = np.transpose(sig.rsos(ifft2c_raki(kspace_residual_est_all),-2),(1,0,2))

truth = np.transpose(sig.rsos(ifft2c_raki(kspace_truth_raki),2),(1,0))

rmse_rraki  = np.zeros(len(all_parameters))
rmse_linear = np.zeros(len(all_parameters))

for pp in range(len(all_parameters)):
    rmse_rraki[pp]  = sig.rmse(sig.nor(truth),sig.nor(all_recons_rraki[:,:,pp])) * 100
    rmse_linear[pp] = sig.rmse(sig.nor(truth),sig.nor(all_recons_linear[:,:,pp])) * 100

### Display a particular reconstruciton 

In [ ]:
pindex = 0

display = sig.nor(np.concatenate((np.expand_dims(all_recons_rraki[:,:,pindex],axis = 0),\
                          np.expand_dims(all_recons_linear[:,:,pindex],axis=0),\
                          np.expand_dims(all_est_residual[:,:,pindex],axis=0)*3),axis = 0))
sig.mosaic(display,1,3)

### saving

In [ ]:
results = {'truth':          np.squeeze(truth),
           'all_rraki' :     np.squeeze(all_recons_rraki),
           'rraki_rmse':     np.squeeze(rmse_rraki),
           'all_linear':     np.squeeze(all_recons_linear),
           'linear_rmse':    np.squeeze(rmse_linear),
           'all_residual':   np.squeeze(all_est_residual),
           'all_parameters': np.squeeze(all_parameters)}

sio.savemat('results/residual_raki_ablation.mat', results, oned_as='row')